In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
from collections import Counter
from sklearn.metrics import mean_absolute_error

In [2]:
working_dir = "/Users/emre/GitHub/HU-AI/AIN313/Assignment 1"
DATASET_PATH = os.path.join(working_dir, "dataset")
PROCESSED_DATASET_PATH = os.path.join(DATASET_PATH, 'processed')
TRAIN_PATH = os.path.join(PROCESSED_DATASET_PATH, 'train.csv')

In [3]:
df = pd.read_csv(TRAIN_PATH)
df.head()

,user_id,anime_id,rating,Duration,Action,Adventure,Avant Garde,Award Winning,Boys Love,Comedy,...,Unknown,Visual novel,Web manga,Web novel,Movie,Music.1,ONA,OVA,Special,TV
0,0,581,0.888889,0.14881,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,10,581,0.888889,0.14881,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,47,581,1.000000,0.14881,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,66,581,0.777778,0.14881,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,82,581,1.000000,0.14881,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
X = df.drop("rating", axis=1)
y = df["rating"]

In [5]:
x_train_folds = np.array_split(X, 5)

y_train_folds = np.array_split(y, 5)

/Users/emre/miniconda3/envs/jupyterenv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/emre/miniconda3/envs/jupyterenv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [6]:
def euc_distance(val_row, train_set):
  return np.linalg.norm(val_row - train_set, axis=1)

In [7]:
def k_nearest_neighbors(k, distances, y_train):
  return Counter(y_train[np.argsort(distances)[:k]]).most_common(1)[0][0]


In [8]:
# Define the kNN algorithm for collaborative filtering
def kNN(k, val_row, train_set, y_train):
    distances = np.linalg.norm(train_set - val_row, axis=1)
    sorted_indices = np.argsort(distances)
    top_k_indices = sorted_indices[:k]
    top_k_ratings = y_train[top_k_indices]
    prediction = np.mean(top_k_ratings)
    return prediction

In [9]:
# Define a function to calculate evaluation metrics
def calculate_metrics(preds, y_value):
    mae = mean_absolute_error(y_value, preds)
    return mae

In [ ]:
def weighted_kNN(k, val_row, train_set, y_train, user_similarity):
    distances = np.linalg.norm(train_set - val_row, axis=1)
    sorted_indices = np.argsort(distances)
    top_k_indices = sorted_indices[:k]
    
    weighted_sum = 0
    weight_sum = 0
    
    for idx in top_k_indices:
        similarity_score = user_similarity[idx]  # Similarity score with the target user
        weighted_sum += similarity_score * y_train[idx]  # Weighted sum of ratings
        weight_sum += similarity_score  # Sum of similarity scores
    
    if weight_sum == 0:
        return 0  # Handle the case where there are no similar users
    
    prediction = weighted_sum / weight_sum
    return prediction


In [10]:
# Set k values for kNN
k_values = [3, 5, 7]

for k in k_values:
    print("For k =", k)

    mae_scores = []

    for i in range(5):
        x_train = x_train_folds.copy()
        val_set = x_train.pop(i).to_numpy()
        train_set = np.concatenate(x_train)

        y_train = y_train_folds.copy()
        y_val_set = y_train.pop(i).to_numpy()
        y_train_set = np.concatenate(y_train)

        preds = []

        for val_row in val_set:
            preds.append(kNN(k, val_row, train_set, y_train_set))

        mae = calculate_metrics(preds, y_val_set)
        mae_scores.append(mae)

    # Calculate the average MAE for this k value
    average_mae = sum(mae_scores) / len(mae_scores)
    print("Average MAE:", average_mae)

For k = 3


Average MAE: 0.1598730415991356
For k = 5


KeyboardInterrupt: 

In [ ]:
# Set k values for kNN
k_values = [3, 5, 7]

for k in k_values:
    print("For weighted k =", k)

    mae_scores = []

    for i in range(5):
        x_train = x_train_folds.copy()
        val_set = x_train.pop(i).to_numpy()
        train_set = np.concatenate(x_train)

        y_train = y_train_folds.copy()
        y_val_set = y_train.pop(i).to_numpy()
        y_train_set = np.concatenate(y_train)

        user_similarity = calculate_user_similarity(train_set)  # Calculate user-user similarity (e.g., cosine similarity)

        preds = []

        for val_row in val_set:
            preds.append(weighted_kNN(k, val_row, train_set, y_train_set, user_similarity))

        mae = calculate_metrics(preds, y_val_set)
        mae_scores.append(mae)

    # Calculate the average MAE for this weighted k value
    average_mae = sum(mae_scores) / len(mae_scores)
    print("Average MAE:", average_mae)